In [15]:
import os
import pandas as pd
import re
import datetime
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file_for_delay_analysis(file_path):
    # Extract person_name and date from the file path
    person_name = os.path.basename(os.path.dirname(file_path))
    folder_date_str = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path))))
    expected_date_minus_one = pd.to_datetime(folder_date_str, format='%Y-%m-%d').date() - datetime.timedelta(days=1)

    delay_messages = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    last_person_msg_time = None
    last_person_msg_index = None

    for i, line in enumerate(lines):
        message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}‚ÄØ[ap]m) - (.*?): (.*)', line)
        if message_match:
            date_time_str, sender, message = message_match.groups()
            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M‚ÄØ%p')

            if date_time.date() != expected_date_minus_one:
                continue

            # Check if the sender is a person (not purely numeric or starting with +)
            is_person = sender is not None and not (sender.strip().isnumeric() or sender.startswith('+'))

            logging.info(f"Sender: {sender}, Is Person: {is_person}, Message: {message}")  # Debug line

            if is_person:
                if last_person_msg_time is not None:
                    # Calculate the time difference in minutes
                    time_diff_seconds = (date_time - last_person_msg_time).total_seconds()
                    time_diff_minutes = time_diff_seconds / 60
                    logging.info(f"Time Difference (minutes): {time_diff_minutes}")  # Debug line
                    if time_diff_minutes > 15:
                        # There is a delay
                        delay = True
                    else:
                        # No delay
                        delay = False
                        time_diff_minutes = None
                else:
                    # No previous message from a person
                    delay = False
                    time_diff_minutes = None
                last_person_msg_time = date_time
                last_person_msg_index = i
            else:
                # Not a message from a person
                delay = False
                time_diff_minutes = None

            if delay:
                # Fetch reference messages
                start_index = max(last_person_msg_index - 3, 0)
                end_index = min(i + 4, len(lines) - 1)
                reference_msgs = lines[start_index:end_index + 1]

                delay_messages.append((date_time, sender, message, delay, time_diff_minutes, reference_msgs))

    return delay_messages, file_path

            
def process_all_files(chat_files):
    all_delay_messages = []

    for file_path in chat_files:
        logging.info(f"Processing file: {file_path}")  # Debug line
        delay_messages, file_path = parse_chat_file_for_delay_analysis(file_path)
        all_delay_messages.extend(delay_messages)

    return all_delay_messages


def create_delay_data_dataframe(all_delay_data):
    columns = ['Date', 'Person', 'Chat Group Name', 'Delay', 'Delay in Mins', 'Reference Messages']
    df_data = []

    for delay_instance in all_delay_data:
        date_time, sender, message, delay, time_diff_minutes, reference_msgs, file_path = delay_instance

        # Use the file name as the chat group name
        chat_group_name = os.path.basename(file_path)

        df_data.append([date_time, sender, chat_group_name, delay, time_diff_minutes, reference_msgs])

    delay_df = pd.DataFrame(df_data, columns=columns)
    return delay_df






# Main Execution
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\filtered_chats"
chat_files = list_chat_files(date_directory)
all_delay_data = process_all_files(chat_files)
delay_analysis_df = create_delay_data_dataframe(all_delay_data)

# Display the first few rows of the DataFrame
print(delay_analysis_df.head())


2023-11-29 16:22:04,954 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Arshita\WhatsApp Chat with Abdul_Edoofa(28_11)AA.txt
2023-11-29 16:22:04,955 - INFO - Sender: Arshita, Is Person: True, Message: 11/20/2023 11:33:18	Abdulrazak Abdullahi 	abdulrazak2abdul@gmail.com	+234906443753	3/27/2001	Male	Al-ihsan mallawa academy 	Private	Nigeria 	Abdullahi shuaibu 	+2347033854402	civil servant 	Aishatu Abdullahi 	+2349036605586	Housewife 	Bachelors Course	chemical engineering 	2019	2021	Yes	Close Friend or Relative.
2023-11-29 16:22:04,956 - INFO - Sender: Arshita, Is Person: True, Message: Source: Nirnoy EWYL23E0617
2023-11-29 16:22:04,956 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:04,957 - INFO - Sender: Arshita, Is Person: True, Message: Add child
2023-11-29 16:22:04,957 - INFO - Time Difference (minutes): 100.0
2023-11-29 16:22:04,958 - INFO - Sender: Arshita, Is Person: True, Message: Greetings ! 
2023-11-29 16:22:04,

2023-11-29 16:22:05,065 - INFO - Sender: +263 71 499 2767, Is Person: False, Message: yes
2023-11-29 16:22:05,065 - INFO - Sender: Arshita, Is Person: True, Message: Greetings! 
2023-11-29 16:22:05,065 - INFO - Time Difference (minutes): 2.0
2023-11-29 16:22:05,065 - INFO - Sender: +263 71 499 2767, Is Person: False, Message: l'm good and you
2023-11-29 16:22:05,065 - INFO - Sender: Arshita, Is Person: True, Message: Good to hear that!
2023-11-29 16:22:05,065 - INFO - Time Difference (minutes): 7.0
2023-11-29 16:22:05,065 - INFO - Sender: Arshita, Is Person: True, Message: I hope you are aware that studying abroad involves a financial aspect. May I know who will support you financially with your education abroad?
2023-11-29 16:22:05,065 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:05,065 - INFO - Sender: +263 71 499 2767, Is Person: False, Message: my parents
2023-11-29 16:22:05,071 - INFO - Sender: Arshita, Is Person: True, Message: That is well noted.
2023-11-29 16:22:05,

2023-11-29 16:22:05,133 - INFO - Time Difference (minutes): 7.0
2023-11-29 16:22:05,133 - INFO - Sender: +263 77 616 3640, Is Person: False, Message: I will try my best.
2023-11-29 16:22:05,133 - INFO - Sender: Arshita, Is Person: True, Message: Caroline, Your final interview is scheduled for *1:00 PM CAT tomorrow(28/11/2023)* Let me share the guidelines with you that you need to keep in mind during the interview.                 
2023-11-29 16:22:05,133 - INFO - Time Difference (minutes): 10.0
2023-11-29 16:22:05,133 - INFO - Sender: Arshita, Is Person: True, Message: Please share the following details for your interview purpose
2023-11-29 16:22:05,133 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:05,133 - INFO - Sender: +263 77 616 3640, Is Person: False, Message: plakedes@92gmail.com
2023-11-29 16:22:05,140 - INFO - Sender: +263 77 616 3640, Is Person: False, Message: Year of intake :2025
2023-11-29 16:22:05,140 - INFO - Sender: Arshita, Is Person: True, Message: Please s

2023-11-29 16:22:05,239 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Arshita\WhatsApp Chat with Elitha_Edoofa(20_11)AA-IE.txt
2023-11-29 16:22:05,256 - INFO - Sender: Arshita, Is Person: True, Message: Greetings!
2023-11-29 16:22:05,257 - INFO - Sender: +263 78 341 4424, Is Person: False, Message: Hi how are you
2023-11-29 16:22:05,258 - INFO - Sender: +263 78 341 4424, Is Person: False, Message: Yes we can begin with the session
2023-11-29 16:22:05,258 - INFO - Sender: Arshita, Is Person: True, Message: Greetings! 
2023-11-29 16:22:05,259 - INFO - Time Difference (minutes): 71.0
2023-11-29 16:22:05,260 - INFO - Sender: Arshita, Is Person: True, Message: Are we still connected?
2023-11-29 16:22:05,260 - INFO - Time Difference (minutes): 85.0
2023-11-29 16:22:05,261 - INFO - Sender: Arshita, Is Person: True, Message: Let us connect at  *11:30 pm CAT on Tuesday(28/11/23) to resume your session (In case you have any challenges or

2023-11-29 16:22:05,296 - INFO - Sender: Arshita, Is Person: True, Message: *6 Cultural Harmony and inclusiveness*- India is a country with diversities in Caste,creed,colour,culture,costume,convictions and cuisines.People come from different backgrounds and there is a Unity in this Diversity that makes India Unique. India ranks second in terms of the Number of English Speaking people in the world making India the most preferred destination for higher education for International Students
2023-11-29 16:22:05,300 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:05,301 - INFO - Sender: Arshita, Is Person: True, Message: Let me also show you some facts about India and global leaders who are the product of Indian Education.
2023-11-29 16:22:05,301 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:05,302 - INFO - Sender: Arshita, Is Person: True, Message: <Media omitted>
2023-11-29 16:22:05,302 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:05,303 - INFO - Sender: Ars

...

2023-11-29 16:22:20,406 - INFO - Sender: +263 77 502 2532, Is Person: False, Message: <Media omitted>
2023-11-29 16:22:20,406 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: Duly received please!
2023-11-29 16:22:20,406 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Shubham Madhwal\WhatsApp Chat with Lydia Mukahiwa_EDOOFA(20_11)GS.txt
2023-11-29 16:22:20,441 - INFO - Sender: +263 77 590 6860, Is Person: False, Message: Network here is hindering progress Sir
2023-11-29 16:22:20,441 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: @263775906860 When would it be convenient for us to connect please?
2023-11-29 16:22:20,441 - INFO - Sender: +263 77 590 6860, Is Person: False, Message: I think when it's midnight here in Zimbabwe
2023-11-29 16:22:20,441 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: @263775906860 I hope you are aware that India is 3 hours 30 minutes ahead of Zimbabwean time as we spe

2023-11-29 16:22:20,670 - INFO - Sender: +263 77 545 0834, Is Person: False, Message: Good morning Sir
2023-11-29 16:22:20,670 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Shubham Madhwal\WhatsApp Chat with Nenyasha Munzararikwa_EDOOFA(24_11)GS.txt
2023-11-29 16:22:20,705 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: This message was deleted
2023-11-29 16:22:20,705 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: Good Afternoon Everyone. 
2023-11-29 16:22:20,705 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:20,705 - INFO - Sender: +263 78 677 2824, Is Person: False, Message: Afternoon sir Yes we may proceed
2023-11-29 16:22:20,705 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: How was your discussion with Mr Gurvinder please?
2023-11-29 16:22:20,705 - INFO - Time Difference (minutes): 8.0
2023-11-29 16:22:20,705 - INFO - Sender: +263 78 677 2824, Is Person: False, Message: The d

2023-11-29 16:22:21,031 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: @263778640240 I didn't get you please?
2023-11-29 16:22:21,031 - INFO - Time Difference (minutes): 384.0
2023-11-29 16:22:21,031 - INFO - Sender: +263 77 864 0240, Is Person: False, Message: How much is the admission fee all in all..
2023-11-29 16:22:21,031 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: You mean after adding the transfer charges, or you meant to ask somthing else please?
2023-11-29 16:22:21,031 - INFO - Time Difference (minutes): 2.0
2023-11-29 16:22:21,031 - INFO - Sender: +263 77 864 0240, Is Person: False, Message: Yes
2023-11-29 16:22:21,031 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: Transfer charges are flat 12.5% on the transaction please, so in total on $350 you would be bearing a transfer charge of $45.
2023-11-29 16:22:21,031 - INFO - Time Difference (minutes): 2.0
2023-11-29 16:22:21,044 - INFO - Sender: +263 77 864 0240, Is Person: False, Message

2023-11-29 16:22:21,165 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Shubham Madhwal\WhatsApp Chat with Takudzwanashe Marange_EA(04_10)TR.txt
2023-11-29 16:22:21,211 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: Good Morning @263788175746
2023-11-29 16:22:21,211 - INFO - Sender: Shubham Madhwal, Is Person: True, Message: Duly received please!
2023-11-29 16:22:21,211 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:22:21,211 - INFO - Sender: +263 78 817 5746, Is Person: False, Message: Okay thank you Sir.
2023-11-29 16:22:21,211 - INFO - Processing file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\filtered_chats\2023-11-28\SALES\Shubham Madhwal\WhatsApp Chat with Tamya Mutematsaka_EDOOFA(07_11)SM23.txt
2023-11-29 16:22:21,229 - INFO - Sender: +27 72 699 5184, Is Person: False, Message: Good morning Mr Shubham
2023-11-29 16:22:21,229 - INFO - Sender: +27 72 699 5184, Is Person: False, Message: Actually l ha

ValueError: not enough values to unpack (expected 7, got 6)

In [ ]:
delay_analysis_df.to_csv("delays.csv", index=False)

In [ ]:
all_delay_data